<a href="https://www.kaggle.com/code/yeemeitsang/titanic-keras?scriptVersionId=110393591" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic-test/test.csv
/kaggle/input/titanic-train/train.csv


In [2]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [3]:
df = pd.read_csv('/kaggle/input/titanic-train/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [5]:
df['Sex'] = df.Sex.replace(['male', 'female'], [0, 1])
df = df.dropna(subset = ['Sex', 'Age', 'Pclass', 'Fare', 'Parch', 'SibSp'])
df.shape

(714, 12)

In [6]:
fare = df.Fare.values.reshape(-1, 1)
fare_scaled = MinMaxScaler().fit_transform(fare)
print(fare_scaled.shape)

(714, 1)


In [7]:
df['Fare'] = pd.DataFrame(fare_scaled)
df['Fare'] = df['Fare'].fillna(value = 0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  714 non-null    int64  
 1   Survived     714 non-null    int64  
 2   Pclass       714 non-null    int64  
 3   Name         714 non-null    object 
 4   Sex          714 non-null    int64  
 5   Age          714 non-null    float64
 6   SibSp        714 non-null    int64  
 7   Parch        714 non-null    int64  
 8   Ticket       714 non-null    object 
 9   Fare         714 non-null    float64
 10  Cabin        185 non-null    object 
 11  Embarked     712 non-null    object 
dtypes: float64(2), int64(6), object(4)
memory usage: 72.5+ KB


In [8]:
X_train = df[['Sex', 'Age', 'Pclass', 'Fare', 'Parch', 'SibSp']]
X_train.shape

(714, 6)

In [9]:
X_train = X_train.astype('float64')
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sex     714 non-null    float64
 1   Age     714 non-null    float64
 2   Pclass  714 non-null    float64
 3   Fare    714 non-null    float64
 4   Parch   714 non-null    float64
 5   SibSp   714 non-null    float64
dtypes: float64(6)
memory usage: 39.0 KB


In [10]:
y_train = df['Survived']
y_train.describe()

count    714.000000
mean       0.406162
std        0.491460
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

In [11]:
df_test = pd.read_csv('/kaggle/input/titanic-test/test.csv')
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [12]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [13]:
df_test['Sex'] = df_test.Sex.replace(['male', 'female'], [0, 1])

In [14]:
fare = df_test.Fare.values.reshape(-1, 1)
fare_scaled = MinMaxScaler().fit_transform(fare)
df_test['Fare'] = pd.DataFrame(fare_scaled)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 36.0+ KB


In [15]:
df_test['Fare'] = df_test.Fare.fillna(value = 0)
df_test['Age'] = df_test.Age.fillna(method = 'pad')

In [16]:
X_test = df_test[['Sex', 'Age', 'Pclass', 'Fare', 'Parch', 'SibSp']].astype('float64')
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Sex     418 non-null    float64
 1   Age     418 non-null    float64
 2   Pclass  418 non-null    float64
 3   Fare    418 non-null    float64
 4   Parch   418 non-null    float64
 5   SibSp   418 non-null    float64
dtypes: float64(6)
memory usage: 19.7 KB


In [17]:
X_test.head()

,Sex,Age,Pclass,Fare,Parch,SibSp
0,0.0,34.5,3.0,0.015282,0.0,0.0
1,1.0,47.0,3.0,0.013663,0.0,1.0
2,0.0,62.0,2.0,0.018909,0.0,0.0
3,0.0,27.0,3.0,0.016908,0.0,0.0
4,1.0,22.0,3.0,0.023984,1.0,1.0


In [18]:
X_train = tf.convert_to_tensor(X_train)
X_test = tf.convert_to_tensor(X_test)
y_train = tf.convert_to_tensor(y_train)
print(type(X_test))

<class 'tensorflow.python.framework.ops.EagerTensor'>


2022-11-08 11:45:02.013703: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [19]:
model = keras.Sequential([
    layers.Dense(128, activation = 'relu', input_shape = (6, )),
    layers.BatchNormalization(),
    layers.Dense(96, activation = 'relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(16, activation = 'relu'),
    layers.Dense(1, activation = 'sigmoid')
])

In [20]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'acc')
es = keras.callbacks.EarlyStopping(monitor = 'val_acc', patience = 100, restore_best_weights = True)
model.fit(X_train, y_train, batch_size = 100, epochs = 400, validation_split = 0.4, callbacks = [es])

2022-11-08 11:45:02.302414: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/400
5/5 [==============================] - 1s 73ms/step - loss: 0.6780 - acc: 0.5818 - val_loss: 0.8051 - val_acc: 0.5909
Epoch 2/400
5/5 [==============================] - 0s 14ms/step - loss: 0.6388 - acc: 0.6168 - val_loss: 0.7001 - val_acc: 0.5909
Epoch 3/400
5/5 [==============================] - 0s 14ms/step - loss: 0.6162 - acc: 0.6846 - val_loss: 0.6629 - val_acc: 0.5909
Epoch 4/400
5/5 [==============================] - 0s 13ms/step - loss: 0.5911 - acc: 0.7664 - val_loss: 0.6421 - val_acc: 0.5909
Epoch 5/400
5/5 [==============================] - 0s 13ms/step - loss: 0.5561 - acc: 0.7897 - val_loss: 0.7033 - val_acc: 0.5944
Epoch 6/400
5/5 [==============================] - 0s 13ms/step - loss: 0.5152 - acc: 0.7897 - val_loss: 0.7609 - val_acc: 0.5944
Epoch 7/400
5/5 [==============================] - 0s 13ms/step - loss: 0.4874 - acc: 0.7967 - val_loss: 0.6769 - val_acc: 0.6119
Epoch 8/400
5/5 [==============================] - 0s 13ms/step - loss: 0.4674 - acc: 0.80

In [21]:
model.save('titanic.h5')

In [22]:
# from keras.models import load_model
# model = load_model('titanic.h5')

In [23]:
prob = model.predict(X_test)
print(prob)

[[9.04501975e-02]
 [3.86684746e-01]
 [1.58073902e-02]
 [9.60724056e-02]
 [8.74095857e-02]
 [4.07113373e-01]
 [3.83821964e-01]
 [1.55493021e-02]
 [4.23677415e-01]
 [2.91463733e-03]
 [8.29762816e-02]
 [2.03109860e-01]
 [9.67420220e-01]
 [1.51716471e-02]
 [9.87474561e-01]
 [7.90354550e-01]
 [9.64086950e-02]
 [8.28528106e-02]
 [2.70216167e-01]
 [2.61794448e-01]
 [1.06970459e-01]
 [8.40963721e-01]
 [9.97887731e-01]
 [3.86802912e-01]
 [9.99285519e-01]
 [1.87609494e-02]
 [9.82851505e-01]
 [6.35091066e-02]
 [3.76661986e-01]
 [1.01394355e-02]
 [3.78550887e-02]
 [1.02950633e-02]
 [9.06854928e-01]
 [9.08105552e-01]
 [5.88404596e-01]
 [1.35068238e-01]
 [4.23011124e-01]
 [4.31710154e-01]
 [6.34061098e-02]
 [8.32294822e-02]
 [1.38897598e-02]
 [4.72974628e-01]
 [9.18188095e-02]
 [8.98606062e-01]
 [9.88465071e-01]
 [6.27661049e-02]
 [2.39713311e-01]
 [7.55381286e-02]
 [9.64948356e-01]
 [9.01795745e-01]
 [5.07403672e-01]
 [4.62357402e-02]
 [6.60610080e-01]
 [9.70501065e-01]
 [4.68231142e-02]
 [2.759397

In [24]:
pred = [0 if x<0.5 else 1 for x in prob]
print(pred)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 

In [25]:
pred = pd.DataFrame(pred, columns = ['Survived'])
sub = pd.concat([df_test.PassengerId, pred], axis=1)
sub.to_csv('submission.csv', index=False)
sub.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
